# Deploying an ensemble model with custom application logic

In the last notebook, we learned how to deploy two models that don't interact.
In this notebook, we'll learn how to compose these models together into an *ensemble* model.
We'll use custom logic expressed directly in Python to determine how to compose the models.

First, we'll start with a simple version that takes half its results from the plot-based recommender and half from the color-based recommender.
Once we have that working, we'll try a more complex version that learns *online* how much of each to return, based on the particular user's preferences.
For example, a user that prefers recommendations based on plot will receive more and more of these with each new request.

## If you haven't finished notebooks 1 and/or 2 yet:

Evaluate the next cell to fill out the missing movie palettes in the local database and deploy a plot and color recommender.
This will let you do this notebook without having finished the first two yet.

In [ ]:
%%bash
bash run_1.sh
bash run_2.sh

## Connect to Ray Serve

First let's set up a connection to the Ray cluster and Serve instance that we started in the last notebook.

In [ ]:
import ray
from ray import serve
from util import MOVIE_IDS
import requests

ray.init(address="auto", ignore_reinit_error=True)
client = serve.connect()

# Some helper methods copied from the previous notebook to ping the color and plot backends.
def send_color_request(movie_id):
    color_handle = client.get_handle("color")
    return ray.get(color_handle.remote(liked_id=MOVIE_IDS[0]))

def send_plot_request(movie_id):
    plot_handle = client.get_handle("plot")
    return ray.get(plot_handle.remote(liked_id=MOVIE_IDS[0]))

## Deploying a static ensemble model

Let's see how we can use the `choose_ensemble_results` helper function to return the right distribution of results from the plot and color recommenders.
This function has 3 return values:

1. The normalized distribution of "color" and "plot" recommendations, computed from the weights given as the first argument.
2. A dict that maps the recommendation type ("color" or "plot") to a list of recommendations of that type that were picked.
3. A list of the concatenated results from the "color" and "plot" recommenders that were given as the second argument. This list is picked based on the normalized distribution.

**Task:** Try playing with the helper function to understand how it works. You can modify the weights given in the first argument and/or the recommendations given in the second argument.

In [ ]:
from util import choose_ensemble_results

color_recs = send_color_request(MOVIE_IDS[0])
plot_recs = send_plot_request(MOVIE_IDS[0])
# Weight color and plot recommendations equally.
choose_ensemble_results({"color": 1, "plot": 1}, {"color": color_recs, "plot": plot_recs})

**Task:** Let's try writing a static ensemble model that fetches results from the color- and plot-based recommenders and combines them with this helper function. Once you've finished the implementation, evaluate the following two cells to deploy the ensemble model and test it out. Here is a skeleton to get you started:

In [ ]:
class ComposedModel:
    def __init__(self):
        # Get handles to the two underlying models.
        client = serve.connect()
        self.color_handle = client.get_handle("color")
        self.plot_handle = client.get_handle("plot")

    def __call__(self, request):
        # TODO: Call the two models and get their predictions.

        # TODO: Combine the results using choose_ensemble_results.Select which results to send to the user based on their clicks.
        #distribution, impressions, chosen = choose_ensemble_results(...)

        return {
#             "dist": distribution,
#             "ids": chosen,
#             "sources": {
#                 i["id"]: source
#                 for source, impression in impressions.items()
#                 for i in impression
#             }
        }

In [ ]:
# Delete the ensemble endpoint and backend, if they exist.
# We wrap it in a try-except block since they shouldn't exist on the first evaluation.
try:
    client.delete_endpoint("ensemble")
    client.delete_backend("ensemble:v0")
except:
    pass

# Create the ensemble backend and endpoint.
# Tip! You can run this cell again if you need to debug the ComposedModel code.
client.create_backend("ensemble:v0", ComposedModel)
client.create_endpoint("ensemble", backend="ensemble:v0", route="/rec/ensemble")

In [ ]:
def send_ensemble_request(movie_id, session_key=None):
    r = requests.get("http://localhost:8000/rec/ensemble", params={"liked_id": MOVIE_IDS[0],
                                                                   "session_key": session_key})
    return r.json()

print(send_ensemble_request(MOVIE_IDS[0]))

## Deploying a custom ensemble model

Now let's try shifting the distribution based on what the user has selected before!
This will involve saving some session state for each user.
Each time the user selects a new movie that they like, we'll record that in their session state.
We'll also record the source of any recommendations that the user likes, either the "plot" or "color" recommender.
Whenever the user selects a movie, we'll use the recorded source to decide how to shift the user's distribution.

On each request to the ensemble model, we'll include a session key that is unique per user which can be used to look up the session state.
We'll use a separate actor to record the mapping from session key to session state.
Let's take a look at how this works.

In [ ]:
from util import ImpressionStore

# Create an actor to store the session state.
impression_store = ImpressionStore.remote()
test_session_key = "session"

# Initially, we have no information about the test user's preferred distribution.
print("Initial distribution:",
      dict(ray.get(impression_store.model_distribution.remote(test_session_key, MOVIE_IDS[0]))))

# Mimic getting some results from the color and plot recommenders.
color_recs = send_color_request(MOVIE_IDS[0])
plot_recs = send_plot_request(MOVIE_IDS[0])
_, impressions, _ = choose_ensemble_results({"color": 1, "plot": 1}, {"color": color_recs, "plot": plot_recs})
# Record the source of recommendations made to the test user.
impression_store.record_impressions.remote(test_session_key, impressions)

# Mimic the user selecting a recommendation based on color.
# Now the distribution will assign color recommendations a weight of 1.
print(
    "Distribution after selecting a movie from the color recommender:",
    dict(ray.get(impression_store.model_distribution.remote(test_session_key, color_recs[0]["id"]))))

**Task:** Modify the ensemble model to update the impression store.
1. Modify the `ComposedModel` class to instantiate an `ImpressionStore` actor, similar to the above cell, in the `__init__` method.
2. Extract the `"session_key"` from the `request` argument, similar to how we extracted the `"liked_id"` in the `ColorRecommender` and `PlotRecommender` classes.
3. Update the `ImpressionStore` actor with each request and use the returned distribution to choose the ensemble results.

Once you've finished these steps, evaluate the next cell to deploy the new backend.
The following cell generates some test requests that chooses recommendations based on color, so that you can see how the distribution changes with each selection.

In [ ]:
# Delete the ensemble endpoint and backend, if they exist.
# We wrap it in a try-except block since they shouldn't exist on the first evaluation.
client.delete_endpoint("ensemble")
client.delete_backend("ensemble:v0")

# Create the ensemble backend and endpoint.
# Tip! You can run this cell again if you need to debug the ComposedModel code.
client.create_backend("ensemble:v0", ComposedModel)
client.create_endpoint("ensemble", backend="ensemble:v0", route="/rec/ensemble")

In [ ]:
response = send_ensemble_request(MOVIE_IDS[0], session_key=test_session_key)
print("Initial recommendation", response)
for movie_id, source in response.items():
    if source == "color":
        break
response = send_ensemble_request(movie_id, session_key=test_session_key)
print("Recommendation after selecting a color recommendation:", response)